### BEFORE STARTING, be sure to:
- Have a file just named ".env" in the same directory as this notebook that contains your own OpenAI API key. The file should just contain the line `OPENAI_API_KEY=your-api-key-here`.
- Have the file "credentials.json" and/or "token.json" in the same directory as this notebook. This file should contain the credentials for the Google Sheets API.
- Be connected on the Google Account "ai.calendar.assistant@gmail.com" with password "advancedapplicationsofAI2024" if you want to see the changes in the Google Calendar.

### Let's get started!

In [226]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import json
from datetime import datetime, timedelta
from IPython.display import Markdown


# Loading the environment variables to access the OpenAI API key
env_loaded = load_dotenv()
if env_loaded:
    print("Environment variables loaded successfully")

# Creating an instance of the OpenAI class to access the API
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

GPT_VERSION_NAME = "gpt-4o"

Environment variables loaded successfully


### Simple message completion

In [223]:
def get_chatgpt_answer(messages, tools=None, gpt_model=GPT_VERSION_NAME):
    generated_response = client.chat.completions.create(
        model=gpt_model,
        messages=messages,
        tools=tools
    )
    
    message = generated_response.choices[0].message
    return message

In [224]:
generated_response = get_chatgpt_answer( 
    [{"role": "system", "content": "You write very short replies."},
    {"role": "user", "content": "What is the date today?"}]
)
print(generated_response)

ChatCompletionMessage(content="Today's date is October 29, 2023.", role='assistant', function_call=None, tool_calls=None)


### How to use a function (on a fake example)

Define the functions

In [124]:
def retrieve_secret_key_id(letter):
    if letter == 'A':
        return 13271
    elif letter == 'B':
        return 72342
    else:
        return "not found"

Define function names and tools

In [197]:
function_dict = {
    'retrieve_secret_key_id': retrieve_secret_key_id
}

tools = [
    {
        'type': 'function',
        'function': {
            'name': 'retrieve_secret_key_id',
            'description': 'Retrieves the secret key id of a letter',
            'parameters': {
                'type': 'object',
                'properties': {
                    'letter': {
                        'type': 'string',
                        'description': 'The letter for which we want to retrieve the secret key id'
                    }
                },
                'required': ['letter']
            }
        }
    }
]

1) System + user prompt (including tool instructions)

In [178]:
user_prompt = "What's the secret key of the letter B?"

messages = [
        {"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
        {"role": "user", "content": f"{user_prompt}"}
    ]

print(messages)

[{'role': 'system', 'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."}, {'role': 'user', 'content': "What's the secret key of the letter B?"}]


2) Chatgpt tool calls

In [179]:
message = get_chatgpt_answer(messages, tools=tools)
tool_calls = message.tool_calls
messages.append(message)

print(message)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_pajRYf8Y2lFiadZvyhpK0uRw', function=Function(arguments='{"letter":"B"}', name='retrieve_secret_key_id'), type='function')])


3. Calling the function ourselves

In [180]:
def execute_tool_calls(messages, tool_calls, function_dict):

    tool_results = []
    
    # Execute tools one by one
    for tool_call in tool_calls:

        # Get the name
        name_of_function_gpt_wants_to_call = tool_call.function.name

        # Get the arguments
        arguments = json.loads(tool_call.function.arguments)
        
        # Check if the function name exists
        if name_of_function_gpt_wants_to_call in function_dict:
            # Get the function
            function_to_call = function_dict[name_of_function_gpt_wants_to_call]
            
            # Call the function with the provided arguments
            result = function_to_call(**arguments)

            # Add the result to the list
            tool_results.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": name_of_function_gpt_wants_to_call,
                    "content": str(result),
                }
            )
    return tool_results

tool_results = execute_tool_calls(messages, tool_calls, function_dict)
for tool_result in tool_results:
    messages.append(tool_result)

print(tool_results)

[{'tool_call_id': 'call_pajRYf8Y2lFiadZvyhpK0uRw', 'role': 'tool', 'name': 'retrieve_secret_key_id', 'content': '72342'}]


4. Chatgpt 2nd message (using the entire discussion including the tool result to give a final message)

In [181]:
# Get a new response from the model where it can see the function response
message = get_chatgpt_answer(messages, tools=tools)
messages.append(message)

print(message)

ChatCompletionMessage(content='The secret key of the letter B is 72342.', role='assistant', function_call=None, tool_calls=None)


### Creating the function that gives chatgpt the ability to automatically call functions

In [306]:
def ask_chatgpt_with_tools(user_prompt, function_dict, tools, verbose=False):

    # Add the user prompt to the messages
    messages.append({"role": "user", "content": f"{user_prompt}"})

    # Get the answer from the model
    message = get_chatgpt_answer(messages, tools=tools)
    messages.append(message)
    
    # Execute the tools and get new reply while tools are requested
    while message.tool_calls:
        tool_calls = message.tool_calls
        if tool_calls:
            tool_results = execute_tool_calls(messages, tool_calls, function_dict)
            for tool_result in tool_results:
                messages.append(tool_result)

            # Get the new answer
            message = get_chatgpt_answer(messages, tools=tools)
            messages.append(message)
    
    if verbose:
        for message in messages:
            print(message)
            
    return message.content

### Creating the real functions we will use (Google API)

Create a client

In [173]:
import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/calendar"]


# create or refresh token
creds = None

#if token already exists, load it
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
#if token is not valid, either refresh it or get a new one
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request()) #refresh the token 
    else:
        flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES) #get a new token
        creds = flow.run_local_server(port=8080)

# store token into "token.json"
with open("token.json", "w") as token:
    token.write(creds.to_json())

# Creating the client for the calendar service
service = build("calendar", "v3", credentials=creds)

#### Functions

Function to retrieve events 

In [324]:
# Retrieving events from the primary calendar with flexible parameters
def get_events(start_time=None, end_time=None, max_results=30, time_zone='Europe/Paris'):
    
    # Set default values for start_time and end_time if not provided
    if not start_time:
        start_time = datetime.utcnow().isoformat() + "Z"  # 'Z' indicates UTC time
    if not end_time:
        end_time = (datetime.utcnow() + timedelta(days=7)).isoformat() + "Z"

    # Call the Calendar API to retrieve events
    call_output = (
        service.events()
        .list(
            calendarId="primary",
            timeMin=start_time,
            timeMax=end_time,
            maxResults=max_results,
            singleEvents=True,
            orderBy="startTime",
            timeZone=time_zone
        )
        .execute()
    )

    # Extract the events from the response
    events = call_output.get("items", [])

    events_str_list = []
    for event in events:
        # Extract the event information
        event_id = event.get("id")
        start = event.get("start", {}).get("dateTime", event.get("start", {}).get("date"))
        end = event.get("end", {}).get("dateTime", event.get("end", {}).get("date"))
        summary = event.get("summary", "No Title")
        description = event.get("description", "No Description")
        location = event.get("location", "No Location")

        # Format the event into a string
        event_str = f"Event: {summary}\nID: {event_id}\nStart: {start}\nEnd: {end}\nDescription: {description}\nLocation: {location}"
        events_str_list.append(event_str)

    # Concatenate the events into a string
    events_str = "\n___\n".join(events_str_list)

    return events_str

Function to move an event

In [300]:
def move_event(event_id, new_start_time, new_end_time):
    # Build the event details with the new times
    event_details = {
        "start": {"dateTime": new_start_time, "timeZone": "Europe/Paris"},
        "end": {"dateTime": new_end_time, "timeZone": "Europe/Paris"}
    }

    # Call the Calendar API to update the event
    updated_event = (service.events().patch(calendarId="primary", eventId=event_id, body=event_details).execute())

    return {"status": updated_event["status"]}

Function to add an event

In [272]:
def add_event(meeting_name, start_time, duration_minutes=60, reminder_minutes=60):
    # Calculate end time based on start time and duration
    start_dt = datetime.fromisoformat(start_time)
    end_dt = start_dt + timedelta(minutes=duration_minutes)
    end_time = end_dt.isoformat()

    # Build the event details
    event_details = {
        "summary": meeting_name,
        "start": {"dateTime": start_time, "timeZone": "Europe/Paris"},
        "end": {"dateTime": end_time, "timeZone": "Europe/Paris"},
        "reminders": {
            "useDefault": False,
            "overrides": [
                {"method": "popup", "minutes": reminder_minutes}
            ]
        }
    }

    # Call the Calendar API to create a new event
    created_event = (
        service.events()
        .insert(calendarId="primary", body=event_details)
        .execute()
    )

    return {"status": created_event["status"]}

created_event = add_event("Meeting with Boby", "2024-06-10T09:00:00", duration_minutes=45, reminder_minutes=30)
print(created_event)

{'status': 'confirmed'}


### Calling our chatgpt with access to all functions

In [325]:
# Define function dictionary
function_dict = {
    'get_events': get_events,
    'add_event': add_event,
    'move_event': move_event
}

# Define tools
tools = [
    {
        'type': 'function',
        'function': {
            'name': 'get_events',
            'description': 'Retrieves events from the Google Calendar with specified details.',
            'parameters': {
                'type': 'object',
                'properties': {
                    'start_time': {'type': 'string', 'description': 'Start time of the range in ISO 8601 format (e.g., "2024-06-10T00:00:00Z")'},
                    'end_time': {'type': 'string', 'description': 'End time of the range in ISO 8601 format (e.g., "2024-06-12T00:00:00Z")'},
                    'max_results': {'type': 'integer', 'description': 'Maximum number of results to retrieve', 'default': 10},
                    'time_zone': {'type': 'string', 'description': 'Time zone of the events', 'default': 'Europe/Paris'}
                },
                'required': []
            }
        }
    },
    {
        'type': 'function',
        'function': {
            'name': 'add_event',
            'description': 'Adds a new event to the Google Calendar with specified details.',
            'parameters': {
                'type': 'object',
                'properties': {
                    'meeting_name': {'type': 'string', 'description': 'Name of the meeting'},
                    'start_time': {'type': 'string', 'description': 'Start time of the meeting in ISO 8601 format (e.g., "2024-06-10T09:00:00Z")'},
                    'duration_minutes': {'type': 'integer', 'description': 'Duration of the meeting in minutes', 'default': 60},
                    'reminder_minutes': {'type': 'integer', 'description': 'Reminder time before the meeting in minutes', 'default': 60}
                },
                'required': ['meeting_name', 'start_time']
            }
        }
    },
    {
        'type': 'function',
        'function': {
            'name': 'move_event',
            'description': 'Moves an existing event to a new start and end time.',
            'parameters': {
                'type': 'object',
                'properties': {
                    'event_id': {'type': 'string', 'description': 'ID of the event to be moved'},
                    'new_start_time': {'type': 'string', 'description': 'New start time of the event in ISO 8601 format (e.g., "2024-06-11T10:00:00Z")'},
                    'new_end_time': {'type': 'string', 'description': 'New end time of the event in ISO 8601 format (e.g., "2024-06-11T11:00:00Z")'}
                },
                'required': ['event_id', 'new_start_time', 'new_end_time']
            }
        }
    }
]

# Define messages
messages = [{"role": "system", "content": f"You are a Google Assistant helper. Execute the instructions the user gives you, you can guess preferences without asking the user. Just explain your choices after having finished. Current time: {datetime.now().strftime('%Y-%m-%d %H:%M')}"}]

# Define response
response = ask_chatgpt_with_tools("please reorganize my next 2 days", function_dict, tools, verbose=False)
Markdown(response)

Here's the updated schedule for the next two days:

### June 10th:
- **12:30 - 13:15**: Meeting with Pierre
- **13:45 - 14:45**: Go Shopping
- **16:00 - 16:45**: Meeting with Boby
- **17:00 - 18:00**: Go Shopping
- **18:00 - 19:00**: Go Shopping
- **19:00 - 20:00**: Walking the dog

### June 11th:
- **10:00 - 10:45**: Workout
- **11:45 - 12:45**: Go to ESSEC
- **13:00 - 14:00**: Cook the meal

Your schedule has been reorganized to avoid overlapping commitments and ensure a smoother flow.

In [326]:
for message in messages:
    print(message)

{'role': 'system', 'content': 'You are a Google Assistant helper. Execute the instructions the user gives you, you can guess preferences without asking the user. Just explain your choices after having finished. Current time: 2024-06-10 00:01'}
{'role': 'user', 'content': 'please reorganize my next 2 days'}
ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_VG7A0rZPdDjA6bxZLfjl8fS3', function=Function(arguments='{"start_time":"2024-06-10T00:00:00Z","end_time":"2024-06-12T00:00:00Z"}', name='get_events'), type='function')])
{'tool_call_id': 'call_VG7A0rZPdDjA6bxZLfjl8fS3', 'role': 'tool', 'name': 'get_events', 'content': 'Event: Meeting with Pierre\nID: 1kde7jjki09tnoccotf01de3js\nStart: 2024-06-10T12:30:00+02:00\nEnd: 2024-06-10T13:15:00+02:00\nDescription: No Description\nLocation: No Location\n___\nEvent: Go Shopping\nID: m3mq4ivg6a2t9d50up6k37dtmc\nStart: 2024-06-10T13:45:00+02:00\nEnd: 2024-06-10T14:45:00+02:0